In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import pickle

2025-02-03 08:26:30.576646: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 08:26:30.687959: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-03 08:26:30.797472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-03 08:26:30.885016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-03 08:26:30.908199: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 08:26:31.082405: I tensorflow/core/platform/cpu_feature_gu

In [2]:
df2 = pd.read_csv('job_descriptions.csv')


In [3]:
df2[['Sector', 'Industry', 'City', 'State', 'Zip', 'Website', 'Ticker', 'CEO']] = df2['Company Profile'].str.extract(
    r'"Sector":"([^"]+)".*?"Industry":"([^"]+)".*?"City":"([^"]+)".*?"State":"([^"]+)".*?"Zip":"([^"]+)".*?"Website":"([^"]+)".*?"Ticker":"([^"]+)".*?"CEO":"([^"]+)"'
)
df2.drop('Company Profile', axis=1, inplace=True)
df2.dropna(inplace=True)
df2.drop('Job Id', axis=1, inplace=True)
# Drop rows with any null or empty values
df2.dropna(inplace=True)

# Get the top 10 sectors by count
top_10_sectors = df2['Sector'].value_counts().nlargest(7).index

# Drop rows that do not have a sector in the top 10 sectors
df2 = df2[df2['Sector'].isin(top_10_sectors)]

label_encoder = LabelEncoder()

# Fit and transform the 'Sector' column
df2['Sector'] = label_encoder.fit_transform(df2['Sector'])

df = df2.iloc[:len(df2)//2].copy()
del df2
df.head()

,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,Job Posting Date,...,Responsibilities,Company,Sector,Industry,City,State,Zip,Website,Ticker,CEO
1,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,2022-12-19,...,"Design and code user interfaces for websites, ...",PNC Financial Services Group,2,Commercial Banks,Pittsburgh,Pennsylvania,15222,www.pnc.com,PNC,William S. Demchak
3,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,2023-02-25,...,"Design, configure, and optimize wireless netwo...",Hess,1,"Mining, Crude-Oil Production",New York,New York,10036,www.hess.com,HES,John B. Hess
4,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,2022-10-11,...,Specialize in conference and convention planni...,Cairn Energy,1,Energy - Oil & Gas Exploration & Production,Edinburgh,Scotland,EH2 4NH,https://www.cairnenergy.com/,CNE,Simon Thomson
13,1 to 8 Years,B.Com,$56K-$86K,Banjul,Gambia,13.4432,-15.3101,Temporary,127900,2022-05-24,...,"Manage and secure computer networks, including...",EnLink Midstream,1,Pipelines,Dallas,Texas,75201,www.enlink.com,ENLC,Jesse Arenivas
14,1 to 9 Years,MCA,$57K-$98K,Damascus,Syrian Arab Republic,34.8021,38.9968,Full-Time,92128,2022-03-01,...,Manage sales accounts and ensure revenue growt...,NGL Energy Partners,1,Pipelines,Tulsa,Oklahoma,74136,www.nglenergypartners.com,NGL,Michael Krimbill


In [4]:
df[['min_exp', 'max_exp']] = df['Experience'].str.split(' to ', expand=True)
df['max_exp'] = df['max_exp'].str.replace(' Years', '')
df.head()

,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,Job Posting Date,...,Sector,Industry,City,State,Zip,Website,Ticker,CEO,min_exp,max_exp
1,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,2022-12-19,...,2,Commercial Banks,Pittsburgh,Pennsylvania,15222,www.pnc.com,PNC,William S. Demchak,2,12
3,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,2023-02-25,...,1,"Mining, Crude-Oil Production",New York,New York,10036,www.hess.com,HES,John B. Hess,4,11
4,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,2022-10-11,...,1,Energy - Oil & Gas Exploration & Production,Edinburgh,Scotland,EH2 4NH,https://www.cairnenergy.com/,CNE,Simon Thomson,1,12
13,1 to 8 Years,B.Com,$56K-$86K,Banjul,Gambia,13.4432,-15.3101,Temporary,127900,2022-05-24,...,1,Pipelines,Dallas,Texas,75201,www.enlink.com,ENLC,Jesse Arenivas,1,8
14,1 to 9 Years,MCA,$57K-$98K,Damascus,Syrian Arab Republic,34.8021,38.9968,Full-Time,92128,2022-03-01,...,1,Pipelines,Tulsa,Oklahoma,74136,www.nglenergypartners.com,NGL,Michael Krimbill,1,9


In [5]:
df.drop('Experience', axis=1, inplace=True)

In [6]:
df[['min_salary', 'max_salary']] = df['Salary Range'].str.split('-', expand=True).replace('K', '000', regex=True)
df.head()

,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,Job Posting Date,Preference,...,City,State,Zip,Website,Ticker,CEO,min_exp,max_exp,min_salary,max_salary
1,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,2022-12-19,Female,...,Pittsburgh,Pennsylvania,15222,www.pnc.com,PNC,William S. Demchak,2,12,$56000,$116000
3,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,2023-02-25,Female,...,New York,New York,10036,www.hess.com,HES,John B. Hess,4,11,$65000,$91000
4,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,2022-10-11,Female,...,Edinburgh,Scotland,EH2 4NH,https://www.cairnenergy.com/,CNE,Simon Thomson,1,12,$64000,$87000
13,B.Com,$56K-$86K,Banjul,Gambia,13.4432,-15.3101,Temporary,127900,2022-05-24,Female,...,Dallas,Texas,75201,www.enlink.com,ENLC,Jesse Arenivas,1,8,$56000,$86000
14,MCA,$57K-$98K,Damascus,Syrian Arab Republic,34.8021,38.9968,Full-Time,92128,2022-03-01,Male,...,Tulsa,Oklahoma,74136,www.nglenergypartners.com,NGL,Michael Krimbill,1,9,$57000,$98000


In [7]:
df['min_salary'] = df['min_salary'].str.replace('$', '')
df['max_salary'] = df['max_salary'].str.replace('$', '')
df.drop('Salary Range', axis=1, inplace=True)
df.head()

,Qualifications,location,Country,latitude,longitude,Work Type,Company Size,Job Posting Date,Preference,Contact Person,...,City,State,Zip,Website,Ticker,CEO,min_exp,max_exp,min_salary,max_salary
1,BCA,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,2022-12-19,Female,Francisco Larsen,...,Pittsburgh,Pennsylvania,15222,www.pnc.com,PNC,William S. Demchak,2,12,56000,116000
3,PhD,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,2023-02-25,Female,Joy Lucero,...,New York,New York,10036,www.hess.com,HES,John B. Hess,4,11,65000,91000
4,MBA,Santiago,Chile,-35.6751,-71.5429,Intern,53944,2022-10-11,Female,Julie Johnson,...,Edinburgh,Scotland,EH2 4NH,https://www.cairnenergy.com/,CNE,Simon Thomson,1,12,64000,87000
13,B.Com,Banjul,Gambia,13.4432,-15.3101,Temporary,127900,2022-05-24,Female,Lisa Franklin,...,Dallas,Texas,75201,www.enlink.com,ENLC,Jesse Arenivas,1,8,56000,86000
14,MCA,Damascus,Syrian Arab Republic,34.8021,38.9968,Full-Time,92128,2022-03-01,Male,Adam White,...,Tulsa,Oklahoma,74136,www.nglenergypartners.com,NGL,Michael Krimbill,1,9,57000,98000


In [8]:
df.drop('Job Posting Date', axis=1, inplace=True)
df.drop('Contact Person', axis=1, inplace=True)
df.drop('Contact', axis=1, inplace=True)

In [9]:
df['Benefits'] = df['Benefits'].str.replace('{', '')
df['Benefits'] = df['Benefits'].str.replace('}', '')
df['Benefits'] = df['Benefits'].str.replace("'", '')
# Split the benefits into separate columns
benefits_split = df['Benefits'].str.get_dummies(sep=', ')

# Concatenate the new columns with the original dataframe
df = pd.concat([df, benefits_split], axis=1)
del benefits_split

# Drop the original Benefits column
df.drop('Benefits', axis=1, inplace=True)

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Initialize the scaler
scaler = StandardScaler()
minscaler = MinMaxScaler()

# Standardize the latitude and longitude columns
df[['latitude', 'longitude']] = scaler.fit_transform(df[['latitude', 'longitude']])

df[['min_salary', 'max_salary', 'min_exp', 'max_exp', 'Company Size']] = minscaler.fit_transform(df[['min_salary', 'max_salary', 'min_exp', 'max_exp', 'Company Size']])

df.drop('Job Portal', axis=1, inplace=True)


In [11]:
from sklearn.preprocessing import OneHotEncoder

# Specify the columns to be one-hot encoded
columns_to_encode = ['Qualifications', 'location', 'Country', 'Work Type', 'Preference', 'Job Title', 'Role', 'Company', 'Industry', 'City', 'State', 'Zip', 'Website', 'Ticker', 'CEO']

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit and transform the specified columns
encoded_columns = encoder.fit_transform(df[columns_to_encode])

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(columns_to_encode))

# Reset index to align properly before concatenation
df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Concatenate the new DataFrame with the original df
df = pd.concat([df, encoded_df], axis=1)
del encoded_columns, encoded_df

# Drop the original columns that were encoded
df.drop(columns_to_encode, axis=1, inplace=True)

df.head()

,latitude,longitude,Company Size,Job Description,skills,Responsibilities,Sector,min_exp,max_exp,min_salary,...,CEO_Warren E. Buffett,CEO_Wei Jianjun,CEO_Wendell P. Weeks,CEO_Will Gardiner,CEO_William C. Pate,CEO_William D. Nash,CEO_William H. Rogers Jr.,CEO_William Li Bin,CEO_William S. Demchak,CEO_Willie Chiang
0,0.830999,0.608316,0.717695,Frontend Web Developers design and implement u...,"HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",2,0.4,0.571429,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.428681,-0.201529,0.959587,"Wireless Network Engineers design, implement, ...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",1,0.8,0.428571,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.339001,-1.246490,0.337982,A Conference Manager coordinates and manages c...,Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,1,0.2,0.571429,0.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.253055,-0.450902,0.943251,Protect an organizations computer networks and...,Network security protocols Intrusion detection...,"Manage and secure computer networks, including...",1,0.2,0.000000,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.654010,0.317438,0.650487,A Sales Account Manager is responsible for bui...,Account management Sales strategy Negotiation ...,Manage sales accounts and ensure revenue growt...,1,0.2,0.142857,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df['latitude'] = df['latitude'].round(2)
df['longitude'] = df['longitude'].round(2)
df['min_salary'] = df['min_salary'].round(2)
df['max_salary'] = df['max_salary'].round(2)
df['min_exp'] = df['min_exp'].round(2)
df['max_exp'] = df['max_exp'].round(2)
df['Company Size'] = df['Company Size'].round(2)

df.head()

,latitude,longitude,Company Size,Job Description,skills,Responsibilities,Sector,min_exp,max_exp,min_salary,...,CEO_Warren E. Buffett,CEO_Wei Jianjun,CEO_Wendell P. Weeks,CEO_Will Gardiner,CEO_William C. Pate,CEO_William D. Nash,CEO_William H. Rogers Jr.,CEO_William Li Bin,CEO_William S. Demchak,CEO_Willie Chiang
0,0.83,0.61,0.72,Frontend Web Developers design and implement u...,"HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",2,0.4,0.57,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.43,-0.20,0.96,"Wireless Network Engineers design, implement, ...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",1,0.8,0.43,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.34,-1.25,0.34,A Conference Manager coordinates and manages c...,Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,1,0.2,0.57,0.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.25,-0.45,0.94,Protect an organizations computer networks and...,Network security protocols Intrusion detection...,"Manage and secure computer networks, including...",1,0.2,0.00,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.65,0.32,0.65,A Sales Account Manager is responsible for bui...,Account management Sales strategy Negotiation ...,Manage sales accounts and ensure revenue growt...,1,0.2,0.14,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

# Download necessary NLTK data
# nltk.download('stopwords')
# nltk.download('wordnet')

# Initialize lemmatizer and stop words

text_columns = ['Job Description', 'skills', 'Responsibilities']
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove unwanted characters, numbers, and symbols
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    # Tokenize and remove stop words
    tokens = [word for word in text.split() if word.lower() not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens

# Apply preprocessing to the text columns
for column in text_columns:
    df[column] = df[column].apply(preprocess_text)

# Combine all text data for Word2Vec training
all_text = df[text_columns].values.flatten()
all_text = [item for sublist in all_text for item in sublist]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=[all_text], vector_size=100, window=5, min_count=1, workers=4)

del all_text
# Function to get the average embedding for a document
def get_average_embedding(tokens, model, embedding_dim=100):
    valid_embeddings = [model.wv[word] for word in tokens if word in model.wv]
    if valid_embeddings:
        return np.mean(valid_embeddings, axis=0)
    else:
        return np.zeros(embedding_dim)

# Apply preprocessing and get embeddings
embedding_dim = 100
for column in text_columns:
    df[column + '_embedding'] = df[column].apply(lambda tokens: get_average_embedding(tokens, word2vec_model, embedding_dim))

# Concatenate the embeddings into a single DataFrame
embeddings_df = pd.DataFrame(df[text_columns[0] + '_embedding'].tolist(), index=df.index)
for column in text_columns[1:]:
    temp_df = pd.DataFrame(df[column + '_embedding'].tolist(), index=df.index)
    embeddings_df = pd.concat([embeddings_df, temp_df], axis=1)
    df = df.drop(column + '_embedding', axis=1)
    del temp_df
if 'temp_df' in locals():
    del temp_df
# Concatenate the new DataFrame with the original df
df = pd.concat([df, embeddings_df], axis=1)
del embeddings_df

# Drop the original columns that were encoded
df.drop(text_columns, axis=1, inplace=True)



df.head()




,latitude,longitude,Company Size,Sector,min_exp,max_exp,min_salary,max_salary,Bonuses and Incentive Programs,Casual Dress Code,...,90,91,92,93,94,95,96,97,98,99
0,0.83,0.61,0.72,2,0.4,0.57,0.1,0.72,0,0,...,0.183272,0.135228,0.026879,0.039166,0.284174,0.184891,0.044333,-0.181131,0.052099,-0.052274
1,-0.43,-0.20,0.96,1,0.8,0.43,1.0,0.22,1,0,...,0.112631,0.082011,0.012784,0.026352,0.173757,0.113633,0.023140,-0.106476,0.026048,-0.035352
2,-2.34,-1.25,0.34,1,0.2,0.57,0.9,0.14,0,0,...,0.083143,0.063033,0.012076,0.019971,0.128245,0.084579,0.018991,-0.078595,0.019393,-0.023368
3,-0.25,-0.45,0.94,1,0.2,0.00,0.1,0.12,1,0,...,0.113728,0.083313,0.014664,0.024468,0.175435,0.112284,0.022770,-0.105440,0.023949,-0.032205
4,0.65,0.32,0.65,1,0.2,0.14,0.2,0.36,0,1,...,0.125324,0.090820,0.013927,0.029202,0.189811,0.124521,0.025556,-0.113828,0.032558,-0.035452


In [14]:
# Check for columns with sequence data types
sequence_columns = df.select_dtypes(include=['object', 'category']).columns

# Print the columns
print(sequence_columns)

Index(['Job Description_embedding'], dtype='object')


In [15]:
df = df.drop('Job Description_embedding', axis=1)

In [16]:
# Convert all integer columns to int8
int_columns = df.select_dtypes(include=['int64', 'int32']).columns
df[int_columns] = df[int_columns].apply(pd.to_numeric, downcast='integer')

# Convert all float columns to float16
float_columns = df.select_dtypes(include=['float64', 'float32']).columns
df[float_columns] = df[float_columns].apply(pd.to_numeric, downcast='float')

# Verify the changes
print(df.dtypes)


ValueError: Columns must be same length as key

In [17]:
del int_columns, float_columns

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import keras_tuner as kt

# Separate features and target
y = df['Sector']  # Target column
df = df.drop('Sector', axis=1)  # Features

# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
del df, y  # Free up memory



# Build Model Function for Hyperparameter Tuning
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units1', min_value=64, max_value=256, step=32), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))
    
    model.add(Dense(hp.Int('units2', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Hyperparameter Tuning with Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  # Maximize validation accuracy
    max_trials=20,  # Number of different models to test
    executions_per_trial=1,  # Runs per model
    directory='my_tuner_results',
    project_name='sector_classification'
)

# Run the search
tuner.search(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Best Number of Units (Layer 1): {best_hps.get('units1')}
Best Dropout (Layer 1): {best_hps.get('dropout1')}
Best Number of Units (Layer 2): {best_hps.get('units2')}
Best Dropout (Layer 2): {best_hps.get('dropout2')}
Best Learning Rate: {best_hps.get('learning_rate')}
""")

# Train the Best Model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate on Test Data
test_loss, test_acc = best_model.evaluate(X_test, y_test)

print(f"Final Test Accuracy: {test_acc:.2f}")

# Save the model
best_model.save('sector_classification_model.h5')


Trial 20 Complete [00h 06m 20s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 5d 11h 40m 32s

Best Number of Units (Layer 1): 128
Best Dropout (Layer 1): 0.2
Best Number of Units (Layer 2): 32
Best Dropout (Layer 2): 0.2
Best Learning Rate: 0.0005



/home/khoulykid/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9446 - loss: 0.1874 - val_accuracy: 1.0000 - val_loss: 4.4208e-08
Epoch 2/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 1.0000 - loss: 1.6472e-04 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 1.0000 - loss: 5.1615e-05 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 1.0000 - loss: 5.2758e-05 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 1.0000 - loss: 1.4186e-05 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 1.0000 - loss: 3.2699e-05 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
6559/6559 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 1.0000 - loss: 3.0691e-05 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
6559/6559 ━━━━━━━━━━

Final Test Accuracy: 1.00


In [ ]:
# List of columns to exclude
text_columns = ['Job Description', 'skills', 'Responsibilities']
# Drop the specified columns
df = df.drop(columns=text_columns)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Sector' column
df['Sector'] = label_encoder.fit_transform(df['Sector'])


# Create a correlation matrix
correlation_matrix = df.corr()

# Display the correlation matrix
correlation_matrix

In [ ]:
# Find columns with correlation higher than 0.8 with the 'Sector' column
high_corr_columns = correlation_matrix['Sector'][correlation_matrix['Sector'].abs() >= 0.7].index

# Print the columns
print(high_corr_columns)